In [1]:
import torch
# The train split of client 1
cl_num = 0
train_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/train.pt')
val_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/val.pt')
test_data_client = torch.load(f'./data/graph_dt/processed/{cl_num}/test.pt')
print(len(train_data_client) + len(val_data_client) + len(test_data_client))


FileNotFoundError: [Errno 2] No such file or directory: './data/graph_dt/processed/5/train.pt'

1: ESOL(12)
2: FreeSolv(11)
3: Lipophilicity (13)
4: BACE (7)
5: BBBP (8)
6: ClinTox (6)
7: MUTAG (1)
8: PTC_MR (3)
9: PTC_MM (2)
10:  PTC_FM (4)
11: PTC_FR (5) Zu viele Datem: 1440
12: NCI109 (10)
13: NCI1 (9)
14: alchemy_full (15)
15: ZINC_full (16)
16: QM9 (14) (zu viele Daten? 130831)

In [ ]:
len(train_data_client)

In [1]:
import os
import sys

from federatedscope.core.cmd_args import parse_args
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.auxiliaries.worker_builder import get_client_cls, get_server_cls
from federatedscope.core.configs.config import global_cfg
from federatedscope.core.fed_runner import FedRunner
from yacs.config import CfgNode
from rdkit.Chem.Draw import IPythonConsole

if os.environ.get('https_proxy'):
    del os.environ['https_proxy']
if os.environ.get('http_proxy'):
    del os.environ['http_proxy']

/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
2022-12-21 11:16:32,972 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [2]:
cfg_file = 'scripts/B-FHTL_exp_scripts/Graph-DT/isolated.yaml'
cfg_client = 'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'
#'scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client.yaml'

init_cfg = global_cfg.clone()
init_cfg.merge_from_file(cfg_file)
# init_cfg.merge_from_list(args.opts)

update_logger(init_cfg)
setup_seed(init_cfg.seed)

# federated dataset might change the number of clients
# thus, we allow the creation procedure of dataset to modify the global cfg object
data, modified_cfg = get_data(config=init_cfg.clone())
init_cfg.merge_from_other_cfg(modified_cfg)

init_cfg.freeze()

# allow different settings for different clients
# cfg_client.merge_from_file(args.cfg_client)
if cfg_client is None:
    cfg_client = None
else:
    cfg_client = CfgNode.load_cfg(open(cfg_client, 'r')).clone()



2022-12-21 11:16:32,999 (utils:129)INFO: the current machine is at 127.0.1.1
2022-12-21 11:16:33,000 (utils:131)INFO: the current dir is /home/michael/Projects/CKIM_Competition
2022-12-21 11:16:33,001 (utils:132)INFO: the output dir is exp/local_gin_on_graph-dt_lr0.1_lstep1_
2022-12-21 11:27:56,666 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-21 11:27:56,667 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-21 11:27:56,686 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 16
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criteri

In [3]:
runner = FedRunner(data=data,
                   server_class=get_server_cls(init_cfg),
                   client_class=get_client_cls(init_cfg),
                   config=init_cfg.clone(),
                   client_config=cfg_client)
_ = runner.run()

/home/michael/Projects/CKIM_other/CIKM22_FL_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2022-12-21 11:27:57,374 (fed_runner:249)INFO: Server #0 has been set up ... 
2022-12-21 11:27:57,378 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-21 11:27:57,379 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-21 11:27:57,398 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 16
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1

track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True


2022-12-21 11:27:57,559 (fed_runner:302)INFO: Client 2 has been set up ... 
2022-12-21 11:27:57,562 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-21 11:27:57,563 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-21 11:27:57,584 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 16
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader:

track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True


2022-12-21 11:27:57,839 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 16
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: CrossEntropyLoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
    min_curv: 0.02
  root: data/
  server_holds_all: False
  shuffle: True
  sizes: [10, 5]
  splits: [0.8, 0.1, 0.1]
  splitter: ooxx
  splitter_args: []
  subsample: 1.0
  target_transform: []
  transform: []
  type: graph-dt
device: 0
distribute:
  use: False
early_stop:
  delta: 0.0
  i

...

track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True


2022-12-21 11:27:58,584 (fed_runner:302)INFO: Client 15 has been set up ... 
2022-12-21 11:27:58,588 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-21 11:27:58,588 (cfg_fl_setting:104)WARNING: In local/global training mode, the sampling related configs are in-valid, we will use all clients. 
2022-12-21 11:27:58,605 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 16
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader

track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True
track_running_stats: True


2022-12-21 11:28:02,999 (client:260)INFO: {'Role': 'Client #2', 'Round': 0, 'Results_raw': {'train_total': 512, 'train_avg_loss': nan, 'train_loss': nan, 'train_imp_ratio': nan}}
2022-12-21 11:28:03,055 (client:260)INFO: {'Role': 'Client #7', 'Round': 0, 'Results_raw': {'train_imp_ratio': -5.063291, 'train_total': 150, 'train_avg_loss': 0.660889, 'train_loss': 99.133352, 'train_acc': 0.6}}
2022-12-21 11:28:03,139 (client:260)INFO: {'Role': 'Client #9', 'Round': 0, 'Results_raw': {'train_imp_ratio': -16.651947, 'train_total': 268, 'train_avg_loss': 0.6829, 'train_loss': 183.017099, 'train_acc': 0.563433}}
2022-12-21 11:28:03,232 (client:260)INFO: {'Role': 'Client #10', 'Round': 0, 'Results_raw': {'train_imp_ratio': 19.212881, 'train_total': 279, 'train_avg_loss': 0.684592, 'train_loss': 191.001037, 'train_acc': 0.544803}}
2022-12-21 11:28:28,022 (client:260)INFO: {'Role': 'Client #14', 'Round': 0, 'Results_raw': {'train_total': 162063, 'train_avg_loss': 0.014177, 'train_loss': 2297.5494

KeyboardInterrupt: 

In [ ]:
1

In [45]:
cfg_client['client_4']['eval']['metrics']=['mse']

In [46]:
cfg_client['client_5']['eval']['metrics']=['mse']

In [47]:
cfg_client['client_6']['eval']['metrics']=['mse']

In [48]:
cfg_client['client_7']['eval']['metrics']=['mse']

In [49]:
cfg_client['client_8']['eval']['metrics']=['mse']

In [50]:
cfg_client['client_9']['eval']['metrics']=['mse']

In [51]:
cfg_client['client_10']['eval']['metrics']=['mse']

In [52]:
cfg_client['client_11']['eval']['metrics']=['mse']

In [53]:
cfg_client['client_12']['eval']['metrics']=['mse']

In [54]:
cfg_client['client_13']['eval']['metrics']=['mse']

In [27]:
cl_list = ['client_' + str(i) for i in range(1,17)]

In [28]:
cl_list

['client_1',
 'client_2',
 'client_3',
 'client_4',
 'client_5',
 'client_6',
 'client_7',
 'client_8',
 'client_9',
 'client_10',
 'client_11',
 'client_12',
 'client_13',
 'client_14',
 'client_15',
 'client_16']

In [30]:
cfg_client['client_13']['train']['local_update_steps']

21

In [31]:
for el in cl_list:
    cfg_client[el]['train']['local_update_steps']=1

In [32]:
cfg_client

CfgNode({'client_1': CfgNode({'model': CfgNode({'out_channels': 1, 'task': 'graphRegression'}), 'criterion': CfgNode({'type': 'MSELoss'}), 'train': CfgNode({'optimizer': CfgNode({'lr': 0.01}), 'local_update_steps': 1})}), 'client_2': CfgNode({'model': CfgNode({'out_channels': 1, 'task': 'graphRegression'}), 'criterion': CfgNode({'type': 'MSELoss'}), 'train': CfgNode({'optimizer': CfgNode({'lr': 0.1}), 'local_update_steps': 1})}), 'client_3': CfgNode({'model': CfgNode({'out_channels': 1, 'task': 'graphRegression'}), 'criterion': CfgNode({'type': 'MSELoss'}), 'train': CfgNode({'optimizer': CfgNode({'lr': 0.1}), 'local_update_steps': 1})}), 'client_4': CfgNode({'model': CfgNode({'out_channels': 2, 'task': 'graphClassification'}), 'criterion': CfgNode({'type': 'CrossEntropyLoss'}), 'train': CfgNode({'optimizer': CfgNode({'lr': 0.01}), 'local_update_steps': 1}), 'eval': CfgNode({'metrics': ['acc']})}), 'client_5': CfgNode({'model': CfgNode({'out_channels': 2, 'task': 'graphClassification'})